In [1]:
import pandas as pd

In [2]:
train_df_1 = pd.read_excel('usual_train.xlsx')
train_df_2 = pd.read_excel('virus_train.xlsx')

In [3]:
test_df_1 = pd.read_excel('usual_eval.xlsx')
test_df_2 = pd.read_excel('virus_eval.xlsx')

In [4]:
train_df = train_df_1.append(train_df_2, ignore_index=True)
test_df = test_df_1.append(test_df_2, ignore_index=True)

In [5]:
train_df['情绪标签'].value_counts()

happy       9802
angry       9666
neural      7209
sad         5639
surprise    2283
fear        1775
Name: 情绪标签, dtype: int64

In [6]:
train_df = train_df.rename(columns={'数据编号':'id', '文本':'content', '情绪标签':'label'})
test_df = test_df.rename(columns={'数据编号':'id', '文本':'content'})

In [7]:
train_df['label'].loc[train_df['label'] == 'happy'] = 0
train_df['label'].loc[train_df['label'] == 'angry'] = 1
train_df['label'].loc[train_df['label'] == 'neural'] = 2
train_df['label'].loc[train_df['label'] == 'sad'] = 3
train_df['label'].loc[train_df['label'] == 'surprise'] = 4
train_df['label'].loc[train_df['label'] == 'fear'] = 5

/Users/lishuqi/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
for i in range(len(train_df)):
    train_df['id'][i] = i
for i in range(len(test_df)):
    test_df['id'][i] = i

/Users/lishuqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lishuqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
train_df['id'] = train_df['id'].astype(int)
train_df['content'] = train_df['content'].astype(str)
train_df['label'] = train_df['label'].astype(int)
test_df['id'] = test_df['id'].astype(int)
test_df['content'] = test_df['content'].astype(str)

In [10]:
test_df['label'] = 2

In [11]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
import os

X = np.array(train_df.index)
y = train_df.loc[:,'label'].to_numpy()

In [12]:
def generate_data(random_state=42):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    i = 0
    for train_index, dev_index in skf.split(X, y):
        print(i, "TRAIN:", train_index, "TEST:", dev_index)
        DATA_DIR = "./data_StratifiedKFold_{}/data_origin_{}/".format(random_state,i)
        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)
        tmp_train_df = train_df.iloc[train_index]
        
        tmp_dev_df = train_df.iloc[dev_index]
        
        test_df.to_csv(DATA_DIR+"test.csv")
        tmp_train_df.to_csv(DATA_DIR + "train.csv")
        tmp_dev_df.to_csv(DATA_DIR+"dev.csv")
        print(tmp_train_df.shape, tmp_dev_df.shape)
        i+=1

In [13]:
generate_data(random_state=24)

0 TRAIN: [    0     1     2 ... 36371 36372 36373] TEST: [   24    26    28 ... 36346 36361 36362]
(29097, 3) (7277, 3)
1 TRAIN: [    0     1     2 ... 36370 36372 36373] TEST: [    3    10    12 ... 36345 36353 36371]
(29098, 3) (7276, 3)
2 TRAIN: [    0     2     3 ... 36366 36370 36371] TEST: [    1     5     6 ... 36369 36372 36373]
(29099, 3) (7275, 3)
3 TRAIN: [    0     1     2 ... 36371 36372 36373] TEST: [    4    35    36 ... 36359 36364 36365]
(29100, 3) (7274, 3)
4 TRAIN: [    1     3     4 ... 36371 36372 36373] TEST: [    0     2     7 ... 36355 36366 36370]
(29102, 3) (7272, 3)


In [14]:
train_df.to_csv('all_train.csv')
test_df.to_csv('all_test.csv')